In [24]:
!pip install datashader

     |████████████████████████████████| 15.5MB 209kB/s 
     |████████████████████████████████| 81kB 12.6MB/s 
     |████████████████████████████████| 1.6MB 56.6MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 665kB 57.5MB/s 
     |████████████████████████████████| 102kB 14.8MB/s 
  Created wheel for datashape: filename=datashape-0.5.2-cp36-none-any.whl size=59430 sha256=78a0c63f4344f4d7532aac46f0f89146f022e26d9327983aaa1edad532f2ec47
  Stored in directory: /root/.cache/pip/wheels/8d/06/05/c1cba3d57bdcfd3960e3f60a9fdc97e4baef2ef09af0ad1ef8
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=68a5f2a378fae7682a546f687317fd1838250c948bba764b316d4b2cb4f8d14e
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=eba018b72038cd9b92612715b592bc98ab8b36a056c66e271b66d

In [25]:
import nltk
from collections import Counter
from nltk import bigrams
from nltk import trigrams
import os
import pandas as pd

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt

import nltk
import numpy as np
import re
import pandas as pd 
import pylab as pl
import matplotlib.pyplot as plt

from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D

from textblob import TextBlob

from matplotlib import pyplot

#plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

import datashader as ds
import datashader.transfer_functions as tf

In [26]:
df = pd.read_csv("/content/qanda_202010222231.csv")
df.head()

,id,company_id,calendar_id,number,executive_id,analyst_id,question,answer,exclude
0,146369,18,163,1,73.0,22,"Thank you, good morning.",Good morning. Good morning.,False
1,146864,23,176,1,293.0,472,"Hi, good morning, guys. I wanted to ask about ...","Hey, John. This is Clark. I'll take that. Kell...",False
2,141372,5,23,1,15.0,62,Group,Good morning Joe. Good morning Joe.,False
3,147314,39,188,1,175.0,21,Good morning. Just looking to get a little bit...,"Yeah, John how are you doing. It's John here. ...",False
4,144010,33,95,1,218.0,3,Good afternoon. Just wanted to start on the br...,That is correct. Yes. We expect savings o...,False


In [27]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [28]:
df['Clean_text'] = np.vectorize(remove_pattern)(df['question'], "@[\w]*")

In [31]:
df['Clean_text'].head()

0                                   Thank good morning
1    good morning guys wanted about loan growth sho...
2                                                Group
3    Good morning Just looking little more color de...
4    Good afternoon Just wanted start branch closur...
Name: Clean_text, dtype: object

In [30]:
df['Clean_text'] = df['Clean_text'].str.replace("[^a-zA-Z#]", " ")
df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [32]:
tokenized_test = df['Clean_text'].apply(lambda x: x.split())

In [34]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_test = tokenized_test.apply(lambda x: [i for i in x]) # stemming
tokenized_test.head()

0                               [Thank, good, morning]
1    [good, morning, guys, wanted, about, loan, gro...
2                                              [Group]
3    [Good, morning, Just, looking, little, more, c...
4    [Good, afternoon, Just, wanted, start, branch,...
Name: Clean_text, dtype: object

In [36]:
for i in range(len(tokenized_test)):
    tokenized_test[i] = ' '.join(tokenized_test[i])

df['Clean_text'] = tokenized_test

In [38]:
df.loc[:,('question','Clean_text')].head()

,question,Clean_text
0,"Thank you, good morning.",Thank good morning
1,"Hi, good morning, guys. I wanted to ask about ...",good morning guys wanted about loan growth sho...
2,Group,Group
3,Good morning. Just looking to get a little bit...,Good morning Just looking little more color de...
4,Good afternoon. Just wanted to start on the br...,Good afternoon Just wanted start branch closur...


In [39]:
tok = []
for i in df['Clean_text']:
    for j in i.split():
        tok.append(j.replace("'","").replace('"',""))
print(tok[0:100])

['Thank', 'good', 'morning', 'good', 'morning', 'guys', 'wanted', 'about', 'loan', 'growth', 'showed', 'good', 'loan', 'growth', 'this', 'quarter', 'wondering', 'could', 'break', 'down', 'little', 'more', 'between', 'commercial', 'side', 'consumer', 'side', 'Specifically', 'commercial', 'some', 'your', 'peers', 'point', 'elevated', 'paydowns', 'notice', 'pure', 'basis', 'your', 'balances', 'were', 'down', 'just', 'wondering', 'what', 'seeing', 'there', 'then', 'mortgage', 'side', 'seemed', 'like', 'resi', 'mortgage', 'driver', 'loan', 'growth', 'this', 'quarter', 'What', 'were', 'some', 'factors', 'there', 'Group', 'Good', 'morning', 'Just', 'looking', 'little', 'more', 'color', 'deposit', 'efforts', 'this', 'quarter', 'what', 'areas', 'what', 'types', 'deposits', 'pushing', 'there', 'broad', 'based', 'increase', 'state', 'rate', 'then', 'what', 'your', 'outlook', 'there', 'there', 'continued', 'push', 'beyond', 'your', 'existing', 'national']


In [17]:
Counter(tok).most_common()

[('the', 25071),
 ('you', 14185),
 ('of', 12766),
 ('to', 11229),
 ('that', 10600),
 ('in', 8368),
 ('a', 8186),
 ('I', 8026),
 ('and', 7130),
 ('on', 6963),
 ('just', 6409),
 ('And', 5271),
 ('is', 4577),
 ('for', 4529),
 ('your', 3977),
 ('about', 3931),
 ('--', 3298),
 ('it', 3079),
 ('what', 2950),
 ('then', 2846),
 ('or', 2844),
 ('think', 2764),
 ('this', 2754),
 ('are', 2706),
 ('we', 2639),
 ('if', 2574),
 ('as', 2492),
 ('how', 2397),
 ('Okay.', 2367),
 ('some', 2304),
 ('at', 2280),
 ('kind', 2278),
 ('have', 2235),
 ('be', 2178),
 ('was', 2140),
 ('youre', 1995),
 ('there', 1882),
 ('more', 1861),
 ('from', 1859),
 ('do', 1821),
 ('with', 1772),
 ('like', 1727),
 ('but', 1674),
 ('going', 1562),
 ('growth', 1537),
 ('little', 1536),
 ('could', 1529),
 ('maybe', 1529),
 ('you.', 1528),
 ('Thank', 1515),
 ('So', 1496),
 ('can', 1463),
 ('loan', 1422),
 ('any', 1405),
 ('Good', 1392),
 ('would', 1311),
 ('good', 1298),
 ('one', 1276),
 ('it.', 1264),
 ('guys', 1234),
 ('bit', 1

In [43]:
def calculate_sentiment(Clean_text):
    return TextBlob(Clean_text).sentiment

In [44]:
def calculate_sentiment_analyser(Clean_text):    
    return analyser.polarity_scores(Clean_text)

In [45]:
df['sentiment']=df.Clean_text.apply(calculate_sentiment)

In [46]:
df['sentiment_analyser']=df.Clean_text.apply(calculate_sentiment_analyser)

In [48]:
df.loc[:,('Clean_text','sentiment','sentiment_analyser')].head(20)

,Clean_text,sentiment,sentiment_analyser
0,Thank good morning,"(0.7, 0.6000000000000001)","{'neg': 0.0, 'neu': 0.156, 'pos': 0.844, 'comp..."
1,good morning guys wanted about loan growth sho...,"(0.17951940035273364, 0.36419753086419754)","{'neg': 0.0, 'neu': 0.774, 'pos': 0.226, 'comp..."
2,Group,"(0.0, 0.0)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Good morning Just looking little more color de...,"(0.25416666666666665, 0.43541666666666673)","{'neg': 0.0, 'neu': 0.841, 'pos': 0.159, 'comp..."
4,Good afternoon Just wanted start branch closur...,"(0.32857142857142857, 0.37857142857142856)","{'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'comp..."
5,good morning Paul hoping just clarify outlook ...,"(0.14629629629629629, 0.2592592592592593)","{'neg': 0.05, 'neu': 0.563, 'pos': 0.387, 'com..."
6,Yeah thanks Good morning everyone Good hear th...,"(0.575, 0.5)","{'neg': 0.133, 'neu': 0.292, 'pos': 0.575, 'co..."
7,Good morning Jamie wondering could give some i...,"(0.12499999999999999, 0.275)","{'neg': 0.0, 'neu': 0.855, 'pos': 0.145, 'comp..."
8,good morning everybody,"(0.7, 0.6000000000000001)","{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'comp..."
9,Good morning guess just follow your comments a...,"(-0.08000000000000003, 0.6533333333333334)","{'neg': 0.062, 'neu': 0.751, 'pos': 0.186, 'co..."


In [52]:
df.to_csv("/content/question_with_sentiment.csv")

In [53]:
s = pd.DataFrame(index = range(0,len(df)),columns= ['compound_score','compound_score_sentiment'])

In [54]:
for i in range(0,len(df)): 
  s['compound_score'][i] = df['sentiment_analyser'][i]['compound']
  
  if (df['sentiment_analyser'][i]['compound'] < 0.2 ):
    s['compound_score_sentiment'][i] = 'Negative'    
  if (df['sentiment_analyser'][i]['compound'] >= 0.5):
    s['compound_score_sentiment'][i] = 'Positive'
  if ((df['sentiment_analyser'][i]['compound'] >= 0.2) & (df['sentiment_analyser'][i]['compound'] < 0.5)):
    s['compound_score_sentiment'][i] = 'Neutral'

In [55]:
df['compound_score'] = s['compound_score']

In [56]:
df['compound_score_sentiment'] = s['compound_score_sentiment']

In [57]:
df.sample(n=20).head(20)

,id,company_id,calendar_id,number,executive_id,analyst_id,question,answer,exclude,Clean_text,sentiment_analyser,sentiment,compound_score,compound_score_sentiment
5494,142978,28,68,16,115.0,270,"Hey, great. Thanks for the question. Phil, in ...",Yeah. So our -- our stated goal is to continue...,False,great Thanks question Phil terms capital talke...,"{'neg': 0.0, 'neu': 0.664, 'pos': 0.336, 'comp...","(0.06931818181818182, 0.49090909090909096)",0.9349,Positive
8582,146808,37,174,26,166.0,92,OK. OK. That sounds good. OK.,"Yes, so that line should go to 0 after the fou...",False,That sounds good,"{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'comp...","(0.7, 0.6000000000000001)",0.4404,Neutral
6917,150090,12,269,21,56.0,106,"Okay. So those salary savings, are they reflec...",There is some in Q2 and then some in Q3 and Q4.,False,Okay those salary savings they reflected reduc...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...","(0.25, 0.25)",0.2263,Neutral
6936,144250,30,104,21,125.0,10,"Okay. And then, as far as the back-office, do ...",We haven't disclosed a number of data centres....,False,Okay then back office give metrics number data...,"{'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'comp...","(0.25, 0.26666666666666666)",0.296,Neutral
1665,141875,17,36,5,NaN,167,"Okay, great. Thanks.",NaN,True,Okay great Thanks,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...","(0.5, 0.48333333333333334)",0.836,Positive
12365,149174,15,243,49,93.0,28,Got you. All right. I appreciate it. Thank you.,Sure.,False,right appreciate Thank,"{'neg': 0.0, 'neu': 0.161, 'pos': 0.839, 'comp...","(0.2857142857142857, 0.5357142857142857)",0.6369,Positive
4265,152928,21,351,13,406.0,76,"Thank you. And as a follow-up, maybe could you...","Well, I think the -- the rewards in the market...",False,Thank follow maybe could just talk little abou...,"{'neg': 0.0, 'neu': 0.645, 'pos': 0.355, 'comp...","(0.20249999999999999, 0.45999999999999996)",0.9531,Positive
3245,153144,25,357,10,154.0,496,Good morning.,"Good morning, Ebrahim.",False,Good morning,"{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'comp...","(0.7, 0.6000000000000001)",0.4404,Neutral
2474,151693,35,316,8,NaN,21,Okay. Got it. Thank you.,NaN,True,Okay Thank,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...","(0.5, 0.5)",0.5267,Positive
9709,147794,27,202,30,142.0,26,"Thank you. And my follow-up question is, we he...",There are a couple of situations that we would...,False,Thank follow question heard loud clear during ...,"{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'comp...","(-0.11172839506172842, 0.354320987654321)",0.8402,Positive


In [58]:
df.to_csv("/content/question_with_sentiment_v2.csv")